In [1]:
print([i/10 for i in range(100)])

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4.0, 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5.0, 5.1, 5.2, 5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6.0, 6.1, 6.2, 6.3, 6.4, 6.5, 6.6, 6.7, 6.8, 6.9, 7.0, 7.1, 7.2, 7.3, 7.4, 7.5, 7.6, 7.7, 7.8, 7.9, 8.0, 8.1, 8.2, 8.3, 8.4, 8.5, 8.6, 8.7, 8.8, 8.9, 9.0, 9.1, 9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9]


In [ ]:
0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.0 1.1 1.2 1.3 1.4 1.5 1.6 1.7 1.8 1.9 2.0 2.1 2.2 2.3 2.4 2.5 2.6 2.7 2.8 2.9 3.0 3.1 3.2 3.3 3.4 3.5 3.6 3.7 3.8 3.9 4.0 4.1 4.2 4.3 4.4 4.5 4.6 4.7 4.8 4.9 5.0 5.1 5.2 5.3 5.4 5.5 5.6 5.7 5.8 5.9 6.0

In [2]:
p = special.hermite(3, monic=False)

In [5]:
p(3.5)

301.00000000000006

In [ ]:
import numpy as np
import tensorflow as tf
from scipy import special

import numpy as np
from scipy.special import hermite

def rho(X, alpha):
    K = len(X)
    rho = 0
    for n in np.ndindex(*[range(K+1) for _ in range(2*K)]):
        n_indices = n[:K]
        m_indices = n[K:]
        alpha_n = np.prod([alpha[i] for i in n_indices])
        alpha_m = np.prod([alpha[i] for i in m_indices])
        H_n = np.prod([hermite(i)(X[j]) for j, i in enumerate(n_indices)])
        H_m = np.prod([hermite(i)(X[j]) for j, i in enumerate(m_indices)])
        rho += alpha_n * np.conj(alpha_m) * H_n * H_m / np.prod([np.sqrt(np.pi*2**(i)*np.math.factorial(i)) for i in n_indices+m_indices])
    return rho   

In [7]:
import tensorflow as tf

def wavefunction(X, alpha, C):
    K = len(X)
    n_indices = tf.range(0,C)
    H_n = tf.stack([tf.math.hermite(i)(X[j]) for j, i in enumerate(n_indices)], axis=-1)
    term1 = tf.exp(-tf.reduce_sum(tf.square(X))/2)
    term2 = tf.math.sqrt(tf.math.pow(tf.constant(np.pi, dtype=tf.float32), 0.5))
    term3 = tf.math.pow(tf.constant(2.0, dtype=tf.float32),n_indices)
    term4 = tf.math.factorial(n_indices)
    denominator = term2*term3*term4
    numerator = term1*H_n
    wave = tf.reduce_prod(numerator/denominator, axis=-1)
    wave = tf.tensordot(wave, alpha, axes=[-1,0])
    return tf.reduce_sum(wave)

In [25]:
import string
K = 2
C = 3
L = 5 

source_idx = list(string.ascii_lowercase[1: K + 1])
#for i in to_trace:
#    source_idx[self.n_qubits + i] = source_idx[i]
#for j in sorted(to_trace + [self.n_qubits + i for i in to_trace], reverse=True):
#    del target_idx[j]
#source_idx = ['a'] + source_idx
#target_idx = ['a'] + target_idx
#einsum_rule = ''.join(source_idx + ['-', '>'] + target_idx)
#partial_traces = np.einsum(einsum_rule, density_matrices.reshape(tuple([density_matrices.shape[0]] + [2 for _ in range(2*self.n_qubits)])))
#partial_traces = partial_traces.reshape((partial_traces.shape[0], 2**len(qubits_to_keep), 2**len(qubits_to_keep)))

In [44]:
import string
from scipy.special import hermite
import numpy as np

K = 2
C = 3
L = 5 

x = np.linspace(-3, 3, L)
alpha = tf.Variable(np.ones(shape=tuple(C for _ in range(K))))

einsum_rule = ''.join([string.ascii_lowercase[1: K + 1]] + [',' + string.ascii_lowercase[i] + 'a' for i in range(1, K+1)] + ['->a'])

hermite_tensor = tf.Variable(np.zeros(shape=(C, L)))

for n in range(C):
    h = hermite(n)
    hermite_tensor[n].assign(h(x))

tf.einsum(einsum_rule, alpha, *(hermite_tensor for _ in range(K)))

<tf.Tensor: shape=(5,), dtype=float64, numpy=array([8.410e+02, 2.500e+01, 1.000e+00, 1.210e+02, 1.681e+03])>

In [42]:
tuple(C for _ in range(K))

(3, 3)